In [27]:
import csv, re, os, sys
from datetime import datetime, timedelta, date

dir_path = './'
in_dir_name = 'in_files'
out_dir_name = 'out_files'

In [28]:
def init_dir():
    dir_list = os.listdir(dir_path)
    is_init = False
    if in_dir_name not in dir_list:
        print('作業ディレクトリが存在しません')
        os.mkdir(dir_path + in_dir_name)
        print('make: /' + in_dir_name)
        is_init = True
    if out_dir_name not in dir_list:
        if not is_init:
            print('作業ディレクトリが存在しません')
        os.mkdir(dir_path + out_dir_name)
        print('make: /' + out_dir_name)
        is_init = True
    if is_init:
        print('初期化が完了しました')
        sys.exit(1)

In [29]:
def get_valid_num(pronpt='', get_int=True, max=100000):
    num = 0
    while True:
        if pronpt != '':
            print(pronpt)
        num = input('>')
        try:
            num = float(num)
            if num > max or num < 0:
                print(f'0以上{max}未満の数を入力してください')
                continue
            else:
                break
        except ValueError:
            print('数字を入力してください')
            continue
    if get_int:
        return int(num)
    else:
        return num

In [ ]:
def format_weather_data():


In [30]:
try:
    init_dir()

    print('使用する機能を選択')
    features = [
        '気象表作成',
        'コースタイム・軌跡分析',
        '概念図用コースタイム作成'
    ]
    for i in range(len(features)):
        print(f'{i}) {features[i]}')
    feature_choice = get_valid_num(pronpt='数字を入力', max=len(features))

    if feature_choice == 0:
        print('0')
    elif feature_choice == 1:
        print('1')
    elif feature_choice == 2:
        print('2')
except SystemExit:
    print('\nプログラムを終了します')
    print('=' * 50)

使用する機能を選択
0) 気象表作成
1) コースタイム・軌跡分析
2) 概念図用コースタイム作成
数字を入力
1
